In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

boto3_sm = boto3.client("sagemaker", region_name="us-east-1")
session = sagemaker.Session()
region = session.boto_session.region_name
bucket = "sagemakerprojectbuc"
print("Using Bucket:", bucket)

/home/mukundan/learning/ai_ml/sih_2024_project/sih_2024/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/01/24 19:52:14] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=754786;file:///home/mukundan/learning/ai_ml/sih_2024_project/sih_2024/.venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=505714;file:///home/mukundan/learning/ai_ml/sih_2024_project/sih_2024/.venv/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/mukundan/.config/sagemaker/config.yaml


[12/01/24 19:52:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=600865;file:///home/mukundan/learning/ai_ml/sih_2024_project/sih_2024/.venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=260858;file:///home/mukundan/learning/ai_ml/sih_2024_project/sih_2024/.venv/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

Using Bucket: sagemakerprojectbuc


In [81]:
df = pd.read_csv("mpc_train.csv")

In [82]:
df.shape

(2000, 21)

In [83]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [84]:
df["price_range"].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [85]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [86]:
# Percentage of values that are missing
df.isnull().sum()*100

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [87]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [88]:
label = features.pop(-1)
label

'price_range'

In [89]:
x = df[features]
y = df[label]

In [90]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [91]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [92]:
print("X_train Shape:", X_train.shape)
print("X_test Shape:", X_test.shape)
print("y_train Shape:", y_train.shape)
print("y_test Shape:", y_test.shape)

X_train Shape: (1600, 20)
X_test Shape: (400, 20)
y_train Shape: (1600,)
y_test Shape: (400,)


In [93]:
trainX = pd.DataFrame(X_train)
trainX['label'] = y_train

testX = pd.DataFrame(X_test)
testX['label'] = y_test

In [94]:
print(trainX.shape)
print(testX.shape)

(1600, 21)
(400, 21)


In [95]:
trainX.to_csv('train_v1.csv', index=False)
testX.to_csv('test_v1.csv', index=False)

In [96]:
# Sagemaker takes training data from S3 bucket, so uploading data to S3 bucket
sk_folder = "sagemaker/mobile_price_classification/sklearncontainer"

train_path = session.upload_data(path="train_v1.csv", bucket=bucket, key_prefix=sk_folder)
test_path = session.upload_data(path="test_v1.csv", bucket=bucket, key_prefix=sk_folder)

In [97]:
print(train_path)

s3://sagemakerprojectbuc/sagemaker/mobile_price_classification/sklearncontainer/train_v1.csv


In [98]:
%%writefile script.py

import argparse
import os
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import joblib
import pathlib
from io import StringIO
import boto3
import pandas as pd
import numpy as np

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ =='__main__':

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=0)
    # parser.add_argument('--epochs', type=int, default=10)
    # parser.add_argument('--batch-size', type=int, default=100)
    # parser.add_argument('--learning-rate', type=float, default=0.1)

    # an alternative way to load hyperparameters via SM_HPS environment variable.
    # parser.add_argument('--sm-hps', type=json.loads, default=os.environ['SM_HPS'])

    # input data and model directories
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    parser.add_argument('--train_file', type=str, default='train_v1.csv')
    parser.add_argument('--test_file', type=str, default='test_v1.csv')

    args, _ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (80%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (20%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model.....")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=None)
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting script.py


In [99]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point = "script.py",
    role="arn:aws:iam::389366992858:role/sagemakerrole",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/anirudhnuti/Library/Application Support/sagemaker/config.yaml


In [100]:
sklearn_estimator.fit({"train":train_path, "test":test_path}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2023-10-15-01-03-29-061


Using provided s3_resource
2023-10-15 01:03:30 Starting - Starting the training job...
2023-10-15 01:03:46 Starting - Preparing the instances for training............
2023-10-15 01:05:42 Downloading - Downloading input data...
2023-10-15 01:06:12 Training - Downloading the training image...
2023-10-15 01:06:53 Uploading - Uploading generated training model2023-10-15 01:06:48,436 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-10-15 01:06:48,439 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-10-15 01:06:48,478 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-10-15 01:06:48,672 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-10-15 01:06:48,684 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-10-15 01:06:48,696 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
202

In [101]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = boto3_sm.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact present at: ", artifact)



2023-10-15 01:07:08 Starting - Preparing the instances for training
2023-10-15 01:07:08 Downloading - Downloading input data
2023-10-15 01:07:08 Training - Training image download completed. Training in progress.
2023-10-15 01:07:08 Uploading - Uploading generated training model
2023-10-15 01:07:08 Completed - Training job completed
Model artifact present at:  s3://sagemaker-us-east-1-389366992858/RF-custom-sklearn-2023-10-15-01-03-29-061/output/model.tar.gz


In [102]:
# Making a copy of the built model for deployment
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::389366992858:role/sagemakerrole",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/anirudhnuti/Library/Application Support/sagemaker/config.yaml


In [103]:
model

In [104]:
# Endpoint Deployment
endpoint_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

EndpointName=Custom-sklearn-model2023-10-15-01-07-53
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/anirudhnuti/Library/Application Support/sagemaker/config.yaml


INFO:sagemaker:Creating model with name: Custom-sklearn-model2023-10-15-01-07-53
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model2023-10-15-01-07-53
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model2023-10-15-01-07-53


-------!

In [105]:
endpoint_name

'Custom-sklearn-model2023-10-15-01-07-53'

In [106]:
# First five rows of the test dataset for prediction using the endpoint created above
testX[features][0:5].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0],
 [1524.0,
  1.0,
  1.8,
  1.0,
  0.0,
  0.0,
  10.0,
  0.6,
  174.0,
  4.0,
  1.0,
  154.0,
  550.0,
  2678.0,
  16.0,
  5.0,
  13.0,
  1.0,
  0.0,
  1.0],
 [1807.0,
  1.0,
  2.1,
  0.0,
  2.0,
  0.0,
  49.0,
  0.8,
  125.0,
  1.0,
  10.0,
  337.0,
  1384.0,
  1906.0,
  17.0,
  13.0,
  13.0,
  0.0,
  1.0,
  1.0],
 [1086.0,
  1.0,
  1.7,
  1.0,
  0.0,
  1.0,
  43.0,
  0.2,
  111.0,
  6.0,
  1.0,
  56.0,
  1150.0,
  3285.0,
  11.0,
  5.0,
  17.0,
  1.0,
  1.0,
  0.0]]

In [107]:
print(predictor.predict(testX[features][0:5].values.tolist()))

[3 0 2 1 3]


In [108]:
# Deleting the endpoint to avoid the costs
boto3_sm.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '2f5b9fb1-829e-4f57-982d-8d95c9b35f88',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2f5b9fb1-829e-4f57-982d-8d95c9b35f88',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 15 Oct 2023 01:49:43 GMT'},
  'RetryAttempts': 0}}